In [ ]:
import sys
sys.path.append('../utils')
from eval import load_diffusion, compute_base_grid
import time
from train_flow_matching import *
import mesh_tools as mt

In [ ]:
BASE_RES = 16
BATCH_SIZE = 10
device='cuda'
SRC = '../../experiments_fm'

In [ ]:
def generate_input(generated_X, sparse_fm):
    with torch.no_grad():
        sparse_fm.model_upsampler.eval()
        input_X = sparse_fm.model_upsampler(
            generated_X, generated_X.trilinear_upsample()).detach()
        times = torch.zeros((input_X.grid_count,), device=generated_X.device).float()
        times = times[input_X.feature.jidx.long()]
        input_X.feature.jdata = sparse_fm.add_x0_noise(input_X)
        return input_X

def generate_level(generated_X, i, example_mesh_name, src, n_steps):
    sparse_fm = load_diffusion(example_mesh_name, i, src)
    sparse_fm.eval()
    t0 = time.time()
    new_XT = generate_input(generated_X, sparse_fm)
    generated_X = sparse_fm.reverse_sample(new_XT, n_steps)
    return DiffusionTensor.from_vdb(generated_X).remove_mask()


In [ ]:
def compute_all_generations(example_mesh_name, src, base_res, max_level=3, eval_batch_size=10, features=10, f_steps = 100, n_steps=100, X0G=None):
    generated_Xs = []
    # blurs = []
    sparse_fm = load_diffusion(example_mesh_name, 0, src)
    sparse_fm.eval()
    if X0G is None:
        X0G = compute_base_grid(example_mesh_name, eval_batch_size, base_res)
    X = grid_to_VDB(X0G, torch.randn, [features])
    t0 = time.time()
    generated_X = sparse_fm.reverse_sample(X, f_steps)
    generated_X = DiffusionTensor.from_vdb(generated_X).remove_mask()

    generated_Xs.append(generated_X)

    for i in range(1, max_level+1):
        generated_X = generate_level(
            generated_X, i, example_mesh_name, src, n_steps)
        generated_Xs.append(generated_X)
    return generated_Xs

In [ ]:
mesh_name = "small-town"

with torch.no_grad():
    GX = compute_all_generations(mesh_name, SRC, BASE_RES, 4, eval_batch_size=BATCH_SIZE, f_steps=200, n_steps=100)

In [ ]:
tens = GX[-1]

for ind in range(BATCH_SIZE):
    disp_d = DiffusionTensor(tens.grid[ind], tens.feature[ind]).remove_mask()
    disp_d.get_global().colored_PC(.08)
    
    # c=(disp_d.feature.jdata[:, 6:9].cpu().detach().numpy()+2)/4
    # plot(*grid_to_mesh(disp_d.grid, colors=c), shading={'wireframe':True})